# Efficiency: writing multi-threaded programs
_COSC 208, Introduction to Computer Systems, 2023-11-27_

## Announcements
* Project 4 due Thurs, Dec 7

## Outline
* Warm-up
* Review: pthreads API
* Creating multiple threads
* Returning values from threads
* Practice writing multi-threaded programs

## Warm-up (Q3 from last class)

* Q1: _What are all possible outputs produced by this program?_

In [3]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'P';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      pthread_join(thread1, NULL);
/*15*/      *ch = 'Q';
/*16*/      pthread_create(&thread2, NULL, &printer, ch);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

I am P
I am Q


<div style="height:10em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Review: pthreads API

Q2: _Consider the following program:_

In [ ]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <string.h>
/* 4*/  void word_count(char *str) {
/* 5*/      int count = 1;
/* 6*/      for (int i = 0; i < strlen(str); i++) {
/* 7*/          if (str[i] == ' ') {
/* 8*/              count++;
/* 9*/          }
/*10*/     }
/*11*/     printf("%d words\n", count);
/*12*/  }
/*13*/  int main(int argc, char *argv[]) {
/*14*/      char *str = "I love CS";
/*15*/      pthread_t thr;
/*16*/      pthread_create(thr, NULL, &word_count, str);
/*17*/      pthread_join(thr);
/*18*/  }

_Compiling this program with `gcc` results in the following errors:_

```
buggy_noreturn.c:16:20: warning: passing argument 1 of ‘pthread_create’ 
makes pointer from integer without a cast [-Wint-conversion]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                    ^~~
      |                    pthread_t {aka long unsigned int}
/usr/include/pthread.h:202:50: note: expected ‘pthread_t * restrict’ 
{aka ‘long unsigned int * restrict’} but argument is of type ‘pthread_t’ 
{aka ‘long unsigned int’}
  202 | extern int pthread_create (pthread_t *__restrict __newthread,
      |                            ~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~
buggy_noreturn.c:16:31: warning: passing argument 3 of ‘pthread_create’ 
from incompatible pointer type [-Wincompatible-pointer-types]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                               ^~~~~~~~~~~
      |                               void (*)(char *)
/usr/include/pthread.h:204:36: note: expected ‘void * (*)(void *)’ but 
argument is of type ‘void (*)(char *)’
  204 |                            void *(*__start_routine) (void *),
      |                            ~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~
buggy_noreturn.c:17:5: error: too few arguments to function ‘pthread_join’
   17 |     pthread_join(thr);
      |     ^~~~~~~~~~~~
/usr/include/pthread.h:219:12: note: declared here
  219 | extern int pthread_join (pthread_t __th, void **__thread_return);
      |            ^~~~~~~~~~~~
```

_How would you change the code to fix these problems?_

* Need to pass `&thr` to `pthread_create` (instead of `thr`) on line 16
* Function executed by a thread must return `void *` and take a single `void *` parameter; replace lines 6-7 with:
    ```C
    void *word_count(void *arg) {
        char *str = (char *)arg;
    ```
    Also add after line 11:
    ```C
    return NULL;
    ```
* Add an additional parameter to `pthread_join` one line 17:
    ```C
    pthread_join(thr, NULL);
    ``` 

<div style="height:3em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Creating multiple threads

* Create an array of `pthread_t` and an array of arguments
* Call `pthread_create` within a loop
* Call `pthread_join` within a separate loop
* Example

In [36]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  #define NUM_THREADS 5
/* 5*/  void *simple(void *arg) {
/* 6*/      int *id = (int *)arg;
/* 7*/      printf("I am thread %d\n", *id);
/* 8*/      return NULL;
/* 9*/  }
/*10*/  int main() {
/*11*/      pthread_t threads[NUM_THREADS];
/*12*/      int ids[NUM_THREADS];
/*13*/      for (int i = 0; i < NUM_THREADS; i++) {
/*14*/          ids[i] = i+1;
/*15*/          pthread_create(threads+i, NULL, &simple, ids+i);
/*16*/      } 
/*17*/      for (int i = 0; i < NUM_THREADS; i++) {
/*18*/          pthread_join(threads[i], NULL);
/*19*/      }
/*20*/      printf("All threads finished\n");
/*21*/  }

I am thread 1
I am thread 2
I am thread 3
I am thread 4
I am thread 5
All threads finished


## Returning values from threads

* _When does a thread end?_ — when the function passed to `pthread_create` finishes (i.e., returns)
* _What happens to a function's parameters and local variables when the function returns?_ — they no longer exist (i.e., the stack frame is destroyed)
* _Where should we store a value that should exist even after a function returns?_ — on the heap
* Need to store a thread's return value on the heap
* Thread returns a pointer to the value on the heap
* Example

In [20]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <string.h>
/* 4*/  #include <pthread.h>
/* 5*/  void *length(void *arg) {
/* 6*/      char *str = (char *)arg;
/* 7*/      int *len = malloc(sizeof(int));
/* 8*/      *len = strlen(str);
/* 9*/      return len;
/*10*/  }
/*11*/  int main() {
/*12*/      pthread_t thread;
/*13*/      char *phrase = "Hello, threads!";
/*14*/      pthread_create(&thread, NULL, &length, phrase);
/*15*/      int *result = NULL;
/*16*/      pthread_join(thread, (void **)&result);
/*17*/      printf("Length: %d\n", *result);
/*18*/      free(result);
/*19*/  }

Length: 15


* `pthread_join` returns `0` if successful, or an error number
* To get the pointer returned by the thread, we need to pass a location where the pointer can be stored — i.e., we need to pass a double pointer

<div style="page-break-after:always;"></div>

## Practice writing multi-threaded programs

* Q3: _Write a function called `sum_array` which takes an array of `ARRAY_LEN` integers and returns the sum of the integers. Your function should have the appropriate prototype/implementation to serve as the entry point for a thread. Assume `ARRAY_LEN` is a constant which has been `#define`d._

<p style="height:20em;"></p>

* Q4: _Write a function called `sum_matrix` which takes an array of `NUM_ARRAYS` arrays of integers (i.e., an `int **`) and returns the sum of all the integers. The function should create `NUM_ARRAYS` threads, each running the `sum_array` function for a single array of integers. Assume `NUM_ARRAYS` is a constant which has been `#define`d._

<div style="page-break-after:always;"></div>

In [2]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

#define ARRAY_LEN 10
#define NUM_ARRAYS 5

void *sum_array(void *args) {
    int *nums = (int *)args;
    int *sum = malloc(sizeof(int));
    *sum = 0;
    for (int i = 0; i < ARRAY_LEN; i++) {
        *sum += nums[i];
    }
    return sum;
}

int sum_matrix(int *matrix[]) {
    pthread_t threads[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        pthread_create(&(threads[i]), NULL, &sum_array, matrix[i]);
    }

    int total = 0;
    for (int i = 0; i < NUM_ARRAYS; i++) {
        int *sum;
        pthread_join(threads[i], (void **)(&sum));
        total += *sum;
        free(sum);
    }

    return total;
}

int main() {
    int *matrix[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        matrix[i] = malloc(sizeof(int) * ARRAY_LEN);
        for (int j = 0; j < ARRAY_LEN; j++) {
            matrix[i][j] = i * 100 + j;
        }
    }

    int sum = sum_matrix(matrix);
    printf("%d\n", sum);
}

10225


## Extra practice

* Q5: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'M';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      *ch = 'N';
/*15*/      pthread_create(&thread2, NULL, &printer, ch);
/*16*/      pthread_join(thread1, NULL);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

```
I am M
I am N
```
OR
```
I am N
I am N
```